In [5]:
import numpy as np

num_fireflies = 20  # Number of fireflies (candidate routing paths)
num_nodes = 10  # Number of nodes in the FANET
fireflies = np.random.randint(0, num_nodes, (num_fireflies, num_nodes))
brightness = np.zeros(num_fireflies)  # Fitness values

# Initialize delay_matrix and pdr_matrix with random values for demonstration.
# Replace this with your actual delay and PDR data.
delay_matrix = np.random.rand(num_nodes, num_nodes)
pdr_matrix = np.random.rand(num_nodes, num_nodes)

def fitness_function(path, delay_matrix, pdr_matrix):
    total_delay = sum(delay_matrix[path[i], path[i+1]] for i in range(len(path)-1))
    total_pdr = np.prod([pdr_matrix[path[i], path[i+1]] for i in range(len(path)-1)])

    # We want to minimize delay and maximize PDR
    return 1 / (total_delay + 1) * total_pdr

def move_firefly(firefly_i, firefly_j, beta=1, alpha=0.2):
    """Move firefly_i towards firefly_j with some randomness."""
    for i in range(len(firefly_i)):
        if np.random.rand() < beta:  # Attraction to brighter firefly
            firefly_i[i] = firefly_j[i]
        firefly_i[i] += alpha * np.random.randint(-1, 2)  # Random movement
    return firefly_i

max_iterations = 50

for _ in range(max_iterations):
    for i in range(num_fireflies):
        brightness[i] = fitness_function(fireflies[i], delay_matrix, pdr_matrix)

    # Move fireflies
    for i in range(num_fireflies):
        for j in range(num_fireflies):
            if brightness[j] > brightness[i]:  # Move i towards j
                fireflies[i] = move_firefly(fireflies[i], fireflies[j])

    best_firefly = fireflies[np.argmax(brightness)]  # Best routing path found

print("Optimized Routing Path:", best_firefly)

Optimized Routing Path: [0 1 1 7 0 1 1 0 3 7]


In [7]:
# FOA Optimized Paths as Initial DQN Experience
best_firefly_paths = [firefly for firefly in fireflies[:5]]  # Take top 5 paths

# Assuming each node in the path can represent a state and the next node as an action:
initial_experience = []
for path in best_firefly_paths:
    for i in range(len(path) - 1):
        state = path[i]
        action = path[i + 1]  # Next node in the path as the action
        # You will need to define how to determine reward and next_state based on your specific DQN logic
        reward = 0  # Placeholder
        next_state = 0  # Placeholder
        initial_experience.append((state, action, reward, next_state))

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define DQN Model
class DQN(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, action_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# Training DQN with FOA Experience
dqn_model = DQN(state_dim=num_nodes, action_dim=num_nodes)
optimizer = optim.Adam(dqn_model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

for state, action, reward, next_state in initial_experience:
    optimizer.zero_grad()
    # Convert state to one-hot encoded tensor
    state_tensor = torch.zeros(num_nodes, dtype=torch.float32)  # Create a tensor of zeros with size num_nodes
    state_tensor[state] = 1  # Set the element corresponding to the state to 1

    action_tensor = torch.tensor(action, dtype=torch.long)

    q_values = dqn_model(state_tensor)
    q_value = q_values[action_tensor]

    loss = loss_fn(q_value, torch.tensor(reward, dtype=torch.float32))
    loss.backward()
    optimizer.step()